# Model Evaluation Exercises

In [136]:
import pandas as pd
import numpy as np
import os
import acquire
import prepare
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report


# visualize
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(11, 9))
plt.rc('font', size=13)

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# acquire
from env import host, user, password
from pydataset import data

### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |

- **In the context of this problem, what is a false positive?**
    - 13 predicted dogs that are actually cats
- **In the context of this problem, what is a false negative?**
    - 7 predicted cats that are actual dogs
- **How would you describe this model?**

In [137]:
TP = 46
FP = 13
FN = 7
TN = 34

In [138]:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f'Accuracy: {Accuracy: .2%}')

Accuracy:  80.00%


### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

### Use the predictions dataset and pandas to help answer the following questions:

In [139]:
df = pd.read_csv('c3.csv')

In [140]:
df.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [141]:
df.shape

(200, 4)

In [142]:
#check for distribution
df.describe()

,actual,model1,model2,model3
count,200,200,200,200
unique,2,2,2,2
top,No Defect,No Defect,No Defect,No Defect
freq,184,190,110,101


In [143]:
#Review value counts to determine baseline
df.actual.value_counts()

No Defect    184
Defect        16
Name: actual, dtype: int64

In [144]:
df['baseline_prediction'] = 'No Defect'

- **An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?**

- Recall will be most appropiate model for this analysis. I will treat defects as positive, and recall will return the precentage of defective ducks that are actually defective. We will use recall because the distribution of defects is not normally distributed.

In [145]:
subset = df[df.actual== "No Defect"]

model1_recall = (subset.model1 == subset.actual).mean()
model2_recall = (subset.model2 == subset.actual).mean()
model3_recall = (subset.model3 == subset.actual).mean()
baseline_recall = (subset.baseline_prediction == subset.actual).mean()

print(f'   model 1 recall: {model1_recall:.2%}')
print(f'   model 2 recall: {model2_recall:.2%}')
print(f'   model 3 recall: {model3_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')

   model 1 recall: 98.91%
   model 2 recall: 55.98%
   model 3 recall: 53.26%
baseline recall: 100.00%


##### - Model 1 would be the best fit model for this use case as it predricts defects at the highest %

#### Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

- Precision would be the best metric here. Again, assuming that a defect is positive, we want to reduce the likelihood of a false positive (we say the duck is defected, give out a vacation to HW, but the duck was not actually defective).

In [146]:
#Precision evaluation metric:

subset = df[df.model1 == 'No Defect']
model_precision = (subset.model1 == subset.actual).mean()

subset = df[df.model2 == 'No Defect']
model2_precision = (subset.model2 == subset.actual).mean()

subset = df[df.model3 == 'No Defect']
model3_precision = (subset.model3 == subset.actual).mean()

subset = df[df.baseline_prediction == 'No Defect']
baseline_precision = (subset.baseline_prediction == subset.actual).mean()

print(f'model precision: {model_precision:.2%}')
print(f'model 2 precision: {model2_precision:.2%}')
print(f'model 3 precision: {model3_precision:.2%}')
print(f'baseline precision: {baseline_precision:.2%}')

model precision: 95.79%
model 2 precision: 93.64%
model 3 precision: 97.03%
baseline precision: 92.00%


### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

#### At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

#### Several models have already been developed with the data, and you can find their results here.

In [147]:
df = pd.read_csv('gives_you_paws.csv')

In [148]:
df.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [149]:
df.shape

(5000, 5)

In [150]:
df.describe()

,actual,model1,model2,model3,model4
count,5000,5000,5000,5000,5000
unique,2,2,2,2,2
top,dog,dog,cat,dog,dog
freq,3254,2937,3212,2508,4253


### **Dog = Positive**
### **Cat = Negative**

- TP = Predicting Dog and actual is Dog
- TN = Predicting Cat and actual is Cat
- FP = Predicting Dog but actual is Cat
- FN = Predicting Cat but actual is Dog


#### Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [173]:
df['baseline_prediction'] = 'dog'

In [174]:
#Peek at the confusion matrix for Model 1 
#Start with Crosstab
pd.crosstab(df.actual, df.model1)

model1,cat,dog
actual,,
cat,1423,323
dog,640,2614


In [179]:
#Model 1 Confusion Matrix
(tn, fp, fn, tp) = confusion_matrix(df.actual, df.model1,
                 labels = ['cat', 'dog']).ravel()
(tn, fp, fn, tp)

(1423, 323, 640, 2614)

In [184]:
#Calculate Accuracy for Model 1 by hand:
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f'Accuracy for Model 1 is {accuracy:.2%}')

Accuracy for Model 1 is 80.74%


- **a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?**

- Models 1 & 4 perform better than the baseline model. Model 1 performs the best.
- Models  2 & 3 perform worse than the baseline model. Model 3 performs the worst.

In [168]:
model_1_accuracy = (df.model1 == df.actual).mean()
model_2_accuracy = (df.model2 == df.actual).mean()
model_3_accuracy = (df.model3 == df.actual).mean()
model_4_accuracy = (df.model4 == df.actual).mean()

baseline_accuracy = (df.baseline_prediction == df.actual).mean()

print(f'   model 1 accuracy: {model_1_accuracy:.2%}')
print(f'   model 2 accuracy: {model_2_accuracy:.2%}')
print(f'   model 3 accuracy: {model_3_accuracy:.2%}')
print(f'   model 4 accuracy: {model_4_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model 1 accuracy: 80.74%
   model 2 accuracy: 63.04%
   model 3 accuracy: 50.96%
   model 4 accuracy: 74.26%
baseline accuracy: 65.08%


- **b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recomend for Phase I? For Phase II?**

- Model 4 would be the best model for a Phase 1 prediction becuase it has the highest recall score which returns all of the dog photos.

- I would then select the model with the best precision score for Phase 2 because I want to minimize the number of False positives. ==> Model 2 has the best precision score.

- **c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recomend for Phase I? For Phase II?**

- Model 2 for Phase 1 because it has the best recall for cat photos. Recall because we want to minimize False negatives to maximize for cat photos. ???Is this a logically correct way to phrase this????
- Model 4 for Phase 2 beacause it has the best precision for cat photos.

### 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

#### Model 1 Evaluation Metrics

In [259]:
y_pred = df.model1
y_true  = df.actual

In [260]:
#Accuracy 
accuracy_score(y_pred, y_true)
print(f'Model 1 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 1 has an accuracy score of 80.74%


In [261]:
#Precision
precision_score(y_true, y_pred, labels=['cat', 'dog'], average=None)

array([0.68977218, 0.89002383])

In [262]:
#Recall
recall_score(y_true, y_pred, labels = ['cat', 'dog'], average = None)

array([0.81500573, 0.80331899])

In [263]:
#Classification Report
target_names = ['cat', 'dog']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84      3254

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000



#### Model 2 Evaluation Metrics

In [264]:
y_pred = df.model2
y_true  = df.actual

In [265]:
#Accuracy
accuracy_score(y_pred, y_true)
print(f'Model 2 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 2 has an accuracy score of 63.04%


In [266]:
#Precision
precision_score(y_true, y_pred, labels=['cat', 'dog'], average=None)

array([0.48412204, 0.89317673])

In [267]:
#Recall
recall_score(y_true, y_pred, labels = ['cat', 'dog'], average = None)

array([0.8906071 , 0.49078058])

In [268]:
#Classification Report
target_names = ['cat', 'dog']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         cat       0.48      0.89      0.63      1746
         dog       0.89      0.49      0.63      3254

    accuracy                           0.63      5000
   macro avg       0.69      0.69      0.63      5000
weighted avg       0.75      0.63      0.63      5000



#### Model 3 Evaluation Metrics

In [244]:
y_pred = df.model3
y_true  = df.actual

In [245]:
#Accuracy
accuracy_score(y_pred, y_true)
print(f'Model 3 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 3 has an accuracy score of 50.96%


In [246]:
#Precision
precision_score(y_true, y_pred, labels=['cat', 'dog'], average=None)

array([0.35834671, 0.65988836])

In [247]:
#Recall
recall_score(y_true, y_pred, labels = ['cat', 'dog'], average = None)

array([0.51145475, 0.50860479])

In [248]:
#Classification Report
target_names = ['cat', 'dog']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         cat       0.36      0.51      0.42      1746
         dog       0.66      0.51      0.57      3254

    accuracy                           0.51      5000
   macro avg       0.51      0.51      0.50      5000
weighted avg       0.55      0.51      0.52      5000



#### Model 4 Evaluation Metrics

In [239]:
y_pred = df.model4
y_true  = df.actual

In [240]:
#Accuracy
accuracy_score(y_pred, y_true)
print(f'Model 4 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 4 has an accuracy score of 74.26%


In [241]:
#Precision
precision_score(y_true, y_pred, labels=['cat', 'dog'], average=None)

array([0.80722892, 0.73124853])

In [242]:
#Recall
recall_score(y_true, y_pred, labels = ['cat', 'dog'], average = None)

array([0.34536082, 0.95574677])

In [243]:
#Classifictaion Report 
target_names = ['cat', 'dog']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         cat       0.81      0.35      0.48      1746
         dog       0.73      0.96      0.83      3254

    accuracy                           0.74      5000
   macro avg       0.77      0.65      0.66      5000
weighted avg       0.76      0.74      0.71      5000

